In [12]:
from new_model.model_utils import resnet_shorten, hybrid_LSTM_training, _classifier, convolutional_block
from keras.layers import LSTM, Reshape, Input
from keras.models import Model
import keras
from training_utils import save_model, DataGenerator, generate_validation_data

import time

# Global constant

In [2]:
IMG_SHAPE = (200, 200, 1)
LSTM_DIM_HIDDEN = 64*2
LEN_SPATIAL_HISTORY = 10
NUM_CLASS = 629

___

# 2.Model Definition

**------------------------**
## 2.1 Encoder

### 2.1.1 Shared encoder

In [3]:
shared_encoder = resnet_shorten(IMG_SHAPE)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [4]:
shared_encoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
conv_0 (Conv2D)                 (None, 100, 100, 32) 832         input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 49, 49, 32)   0           conv_0[0][0]                     
__________________________________________________________________________________________________
bn_1_a (BatchNormalization)     (None, 49, 49, 32)   128         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
activation

**------------------------**
## 2.2 Decoder

### 2.2.1 Define LSTM

In [5]:
LSTM_cell = LSTM(LSTM_DIM_HIDDEN, return_state=True)

### 2.2.2 Define Classifier

In [6]:
classifier = _classifier(input_shape=(LSTM_DIM_HIDDEN, ), num_class=NUM_CLASS)

In [7]:
classifier.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               33024     
_________________________________________________________________
dense_2 (Dense)              (None, 629)               161653    
_________________________________________________________________
activation_5 (Activation)    (None, 629)               0         
Total params: 194,677
Trainable params: 194,677
Non-trainable params: 0
_________________________________________________________________


In [8]:
reshapor = Reshape((1, -1))

## 2.2 Full Model

In [16]:
flattener = keras.layers.Flatten()
activator = keras.layers.Activation('relu')


def model_shared_private_encoder(image_shape, shared_encoder, LSTM_cell, LSTM_dim_hidden_state, Ty=10):
    """
    Define full model with both shared & private encoder
    
    Input:
        image_shape (tuple): shape of input image
        shared_encoder (keras.Model): shared model used to extract low level feature vector from input image 
        LSTM_cell (keras.layers): shared LSTM layer
        LSTM_dim_hidden_state (int): dimension of LSTM_cell's hidden state
        Ty (int): length of spatial history
    
    Output:
        keras Model instance
    """
    # Input layer
    X_input_list = [Input(shape=image_shape) for i in range(Ty)]
    
    # pass each input through shared encoder
    shared_encoded_X = [shared_encoder(X) for X in X_input_list]
    
    # pass each encoded_X through its own convolution block
    private_encoded_X = [convolutional_block(X, [128, 128, 128], [3, 3, 1], [2, 1, 2], stage=30+i) 
                         for i, X in enumerate(shared_encoded_X)]
    
    # initialize input & cell state
    a_0 = Input(shape=(LSTM_dim_hidden_state, ))  
    c_0 = Input(shape=(LSTM_dim_hidden_state, ))
    
    a = a_0
    c = c_0
    
    outputs = []
    
    # Decode
    for encoded_X in private_encoded_X:
        # flatten & activate encoded_X 
        X = flattener(encoded_X)
        X = activator(X)
        
        # perform 1 step of LSTM cell
        X = reshapor(encoded_X)
        a, _, c = LSTM_cell(X, initial_state=[a, c])
        
        # apply regressor to the hidden state of LSTM_cell
        out = classifier(a)
        
        # append out to outputs
        outputs.append(out)
    
    # define model
    model = Model(inputs=X_input_list + [a_0, c_0], outputs=outputs)
    
    return model


In [17]:
hybrid_model = model_shared_private_encoder(IMG_SHAPE, shared_encoder, LSTM_cell, LSTM_DIM_HIDDEN, Ty=10)

In [18]:
hybrid_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 200, 200, 1)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 13, 13, 64)   78560       input_13[0][0]                   
                                                                 input_14[0][0]                   
                                                                 input_15[0][0]                   
                                                                 input_16[0][0]                   
                                                                 input_17[0][0]                   
                                                                 input_18[0][0]                   
          

In [19]:
# Load predefine weights
shared_encoder.load_weights("./new_model/weights/encoder_resnet8_2019_05_21_10_30.h5", by_name=True)

In [20]:
otim = keras.optimizers.Adam(lr=0.25, decay=0.001)
hybrid_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

---

# 3. Training

In [21]:
param = {'img_shape': IMG_SHAPE, 
         'Ty': LEN_SPATIAL_HISTORY, 
         'num_class': NUM_CLASS, 
         'batch_size': 5, 
         'shuffle': True, 
         'additional_input_for_LSTM': True, 
         'LSTM_dim_hidden_states': LSTM_DIM_HIDDEN}

time_str = time.strftime("%Y_%m_%d_%H_%M")
tb_callback = keras.callbacks.TensorBoard(log_dir='./logs/' + time_str, histogram_freq=0, batch_size=32, 
                                          write_graph=False, write_grads=False, write_images=True, 
                                          update_freq=5000)

training_generator = DataGenerator("./new_data/new_hybrid_training_CH2_only.csv", **param)
X_val, y_val = generate_validation_data("./new_data/new_hybrid_validation_CH2_only.csv", IMG_SHAPE, NUM_CLASS, 
                                        LEN_SPATIAL_HISTORY, LSTM_DIM_HIDDEN)

In [22]:
hybrid_model.fit_generator(training_generator,
                           validation_data=(X_val, y_val),
                           epochs=3,
                           callbacks=[tb_callback])

Instructions for updating:
Use tf.cast instead.
Epoch 1/3
3642/3642 [==============================] - 856s 235ms/step - loss: 49.7892 - model_2_loss: 4.9702 - model_2_acc: 0.0980 - model_2_acc_1: 0.0955 - model_2_acc_2: 0.0967 - model_2_acc_3: 0.0962 - model_2_acc_4: 0.0960 - model_2_acc_5: 0.0948 - model_2_acc_6: 0.0938 - model_2_acc_7: 0.0927 - model_2_acc_8: 0.0937 - model_2_acc_9: 0.0932 - val_loss: 50.3461 - val_model_2_loss: 5.0053 - val_model_2_acc: 0.0934 - val_model_2_acc_1: 0.0968 - val_model_2_acc_2: 0.0983 - val_model_2_acc_3: 0.0958 - val_model_2_acc_4: 0.0949 - val_model_2_acc_5: 0.0944 - val_model_2_acc_6: 0.0958 - val_model_2_acc_7: 0.0963 - val_model_2_acc_8: 0.1023 - val_model_2_acc_9: 0.1013
Epoch 2/3
3642/3642 [==============================] - 843s 232ms/step - loss: 49.1348 - model_2_loss: 4.9201 - model_2_acc: 0.1011 - model_2_acc_1: 0.0993 - model_2_acc_2: 0.1005 - model_2_acc_3: 0.1004 - model_2_acc_4: 0.1012 - model_2_acc_5: 0.0991 - model_2_acc_6: 0.0972 - m

In [ ]:
# save encoder weights
encoder.save_weights("./new_model/weights/encoder_resnet8_" + time_str + ".h5")